In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas_datareader.data as web
import FinanceDataReader as fdr
from sklearn import neighbors,svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import FinanceDataReader as fdr
import mglearn
from sklearn.preprocessing import MinMaxScaler
import sklearn

#성과 평가
from sklearn.metrics import confusion_matrix #오차행렬
from sklearn.metrics import accuracy_score #정확도
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve

setlen=2000


C:\Users\이승희\AppData\Roaming\Python\Python36\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\이승희\AppData\Roaming\Python\Python36\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
    # 3중 분류


KS=fdr.DataReader("SSEC","2009-01-01","2019-08-14")

#특성 추가---
KS=get_MA(KS)
KS=get_stochastic(KS)
KS=get_time(KS)

X=KS
#--------

# 3중 분류를 위한 종가 스케일 조정 ---------------
X_Change=X["Change"].values.reshape(-1,1)
list=[]
    for x in X_Change:
        if x <-0.01:
            list.append(0)
        elif -0.01<=x<=0.01:
            list.append(1)
        elif 0.01<x:
            list.append(2)
y=list
#-----------------------------------------


#미래 예측을 위한 시간조정 ------------------- 
y=pd.DataFrame(np.array(y),index=KS.index).shift(-1).dropna()
X=X.loc[y.index]
#----------------------------
setlen=2000
X_train,X_test,y_train,y_test=X[:setlen],X[setlen:],y[:setlen],y[setlen:]

reg=neighbors.KNeighborsClassifier(n_neighbors=4).fit(X_train,y_train)
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))

confusion=confusion_matrix(y_test,reg.predict(X_test))
confusion

NameError: name 'get_MA' is not defined

In [ ]:


#--- 스케일 조정
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

StandardScaler = MinMaxScaler().fit(X) 
X_scaler= StandardScaler.transform(X)
X=pd.DataFrame(X_scaler,index=y.index,columns=X.columns)
#- 차원 축소

#pca=PCA(n_components=2)
#pca.fit(X_scaler)

#X=pd.DataFrame(X_scaler,index=X.index,columns=KS.columns)
#X_pca=pca.transform(X_scaler)
#X_train,X_test,y_train,y_test=X_pca[:setlen],X_pca[setlen:],y[:setlen],y[setlen:]
#-----------------------
X_train,X_test,y_train,y_test=X[:setlen],X[setlen:],y[:setlen],y[setlen:]

In [ ]:
#특성 추가 ------------------------------------------
    #이동평균선
def get_MA(df):
    MA_10=df["Close"].rolling(10).mean()
    MA_60=df["Close"].rolling(60).mean()
    df=df.assign(MA_10=MA_10,MA_60=MA_60).dropna()
    
    return df

    
    #스토캐스틱
def get_stochastic(df, n=15, m=5, t=3):
    # n일중 최고가
    ndays_high = df.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.Close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df
   
    #시간
def get_time(df):
    time=np.linspace(0,10,len(df),endpoint=False).reshape(-1,1)
    df=df.assign(time=time)
    
    return df

In [ ]:
#이중 분류
KS=fdr.DataReader("SSEC","2009-01-01","2019-08-14")
time=np.linspace(0,10,len(KS),endpoint=False).reshape(-1,1)

#특성 추가---
KS=get_MA(KS)
KS=get_stochastic(KS)
KS=get_time(KS)
X=KS
#--------


# 이중 분류, 미래 예측을 위한 시간조정 ------------------- 

hist_data = plt.hist(KS['Change'],bins = 5)
df = pd.DataFrame(index = KS.index, columns = ['Y'])

initial = hist_data[1][0]
num = 0
for i in hist_data[1][1:]:
    df.loc[KS[KS['Change'] > initial][KS['Change'] < i].index] = num
    initial = i
    num += 1
y=df  

#미래 예측을 위한 시간조정 ------------------- 
y=pd.DataFrame(np.array(y),index=KS.index).shift(-1).dropna()
X=X.loc[y.index]

#----------------------------


X_train,X_test,y_train,y_test=X[:setlen],X[setlen:],y[:setlen],y[setlen:]

In [ ]:
reg=neighbors.KNeighborsClassifier(n_neighbors=2).fit(X_train,y_train)
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))

confusion=confusion_matrix(y_test,reg.predict(X_test))
print(confusion)

In [ ]:
reg=RandomForestClassifier(n_estimators=4).fit(X_train,y_train)
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))

confusion=confusion_matrix(y_test,reg.predict(X_test))
print(confusion)

In [ ]:
reg=svm.SVC(C=10,gamma=0.1).fit(X_train,y_train)
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))

confusion=confusion_matrix(y_test,reg.predict(X_test))
print(confusion)

In [ ]:
reg=GradientBoostingClassifier(random_state=0).fit(X_train,y_train)
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))

confusion=confusion_matrix(y_test,reg.predict(X_test))
print(confusion)

In [ ]:
# 다중 분류(개수가 많으면 안되는 것 같음)
#외삽 풀리는 KS11(코스피) KQ11(코스닥) SSEC(상해) HSCE(홍콩) RTSI(러시아)


#다중 3개

KS=fdr.DataReader("SSEC","2009-01-01","2019-08-14")
time=np.linspace(0,10,len(KS),endpoint=False).reshape(-1,1)

#특성 추가---
KS=get_MA(KS)
KS=get_stochastic(KS)
KS=get_time(KS)

X=KS
#--------

# 3중 분류를 위한 종가 스케일 조정 ---------------
X_Close=X["Close"].values.reshape(-1,1)
minMaxScaler = MinMaxScaler().fit(X_Close)
X_trans= minMaxScaler.transform(X_Close)
if X_trans
            
y=list
#-----------------------------------------


#미래 예측을 위한 시간조정 ------------------- 
y=pd.DataFrame(np.array(y)*20,index=KS.index).shift(-1).dropna()
X=X.loc[y.index]
#----------------------------



#--- 스케일 조정
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

StandardScaler = MinMaxScaler().fit(X) 
X_scaler= StandardScaler.transform(X)
X=pd.DataFrame(X_scaler,index=y.index,columns=X.columns)
#- 차원 축소

#pca=PCA(n_components=2)
#pca.fit(X_scaler)

#X=pd.DataFrame(X_scaler,index=X.index,columns=KS.columns)
#X_pca=pca.transform(X_scaler)
#X_train,X_test,y_train,y_test=X_pca[:setlen],X_pca[setlen:],y[:setlen],y[setlen:]
#-----------------------
X_train,X_test,y_train,y_test=X[:setlen],X[setlen:],y[:setlen],y[setlen:]

In [ ]:
​

#X=pd.DataFrame(X_scaler,index=X.index,columns=KS.columns)
#X_pca=pca.transform(X_scaler)
#X_train,X_test,y_train,y_test=X_pca[:setlen],X_pca[setlen:],y[:setlen],y[setlen:]
#-----------------------
X_train,X_test,y_train,y_test=X[:setlen],X[setlen:],y[:setlen],y[setlen:]



In [ ]:
reg=neighbors.KNeighborsClassifier(n_neighbors=2).fit(X_train,y_train)
plt.plot(X.index,y,'go')
plt.plot(X[:setlen].index,reg.predict(X_train),'b.')
plt.plot(X[setlen:].index,reg.predict(X_test),'y.')
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))
print("이웃-2")

confusion=confusion_matrix(y_test,reg.predict(X_test))
print(confusion)

In [ ]:
reg=RandomForestClassifier(n_estimators=50).fit(X_train,y_train)
plt.plot(X.index,y,'go')
plt.plot(X[:setlen].index,reg.predict(X_train),'b.')
plt.plot(X[setlen:].index,reg.predict(X_test),'y.')
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))
print("랜덤 포레스트")

confusion=confusion_matrix(y_test,reg.predict(X_test))
print(confusion)

In [ ]:
reg=svm.SVC(C=1000,gamma=0.001).fit(X_train,y_train)
plt.plot(X.index,y,'go')
plt.plot(X[:setlen].index,reg.predict(X_train),'b')
plt.plot(X[setlen:].index,reg.predict(X_test),'y')
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))
print("SVM")

confusion=confusion_matrix(y_test,reg.predict(X_test))
print(confusion)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
reg=GradientBoostingClassifier(random_state=0).fit(X_train,y_train)
plt.plot(X.index,y,'go')
plt.plot(X[:setlen].index,reg.predict(X_train),'b')
plt.plot(X[setlen:].index,reg.predict(X_test),'y')
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))
print("그레이디언트 부스팅")

confusion=confusion_matrix(y_test,reg.predict(X_test))
print(confusion)

In [ ]:
reg=LogisticRegression().fit(X_train,y_train)
plt.plot(X.index,y,'go')
plt.plot(X[:setlen].index,reg.predict(X_train),'b')
plt.plot(X[setlen:].index,reg.predict(X_test),'y')
print("훈련 세트 정확도:{:.3f}".format(reg.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(reg.score(X_test,y_test)))
print("SVM")

confusion=confusion_matrix(y_test,reg.predict(X_test))
print(confusion)

In [ ]:
#그리드서치 간단모듈
from sklearn.model_selection import GridSearchCV

param_grid={'random_state':[2,4,8,16,32,64]}
grid_search=GridSearchCV(GradientBoostingClassifier(),param_grid,cv=5,return_train_score=True)
grid_search.fit(X_train,y_train)
grid_search.best_estimator_


In [ ]:
#히트맵
results=pd.DataFrame(grid_search.cv_results_)
scores=np.array(results.mean_test_score).reshape(6,1)
mglearn.tools.heatmap(scores,xlabel='length',xticklabels='',ylabel='random_state',yticklabels=param_grid['random_state'])

In [ ]:
#그리드서치 간단모듈
from sklearn.model_selection import GridSearchCV

param_grid={'n_estimators':[10,50,100,120,300]}
grid_search=GridSearchCV(RandomForestClassifier(),param_grid,cv=5,return_train_score=True)
grid_search.fit(X_train,y_train)
grid_search.best_estimator_
#히트맵
results=pd.DataFrame(grid_search.cv_results_)
scores=np.array(results.mean_test_score).reshape(5,1)
mglearn.tools.heatmap(scores,xlabel='length',xticklabels='',ylabel='n_estimators',yticklabels=param_grid['n_estimators'])


In [ ]:

reg1=neighbors.KNeighborsClassifier(n_neighbors=2).fit(X_train,y_train).predict(X_test)
reg2=svm.SVC(C=1000,gamma=0.01).fit(X_train,y_train).predict(X_test)
reg3=RandomForestClassifier(n_estimators=50).fit(X_train,y_train).predict(X_test)
reg4=GradientBoostingClassifier(random_state=0).fit(X_train,y_train).predict(X_test)

print("최근접 이웃 모델의 f1 score : {:.2f}".format(f1_score(y_test,reg1)))
print("SVM 모델의 f1 score : {:.2f}".format(f1_score(y_test,reg2)))
print("랜덤 포레스트 모델의 f1 score : {:.2f}".format(f1_score(y_test,reg3)))
print("그레이디언트 부스팅 모델의 f1 score : {:.2f}".format(f1_score(y_test,reg4)))

In [ ]:
print("이중 분류")
print(classification_report(y_test,reg4,target_names=["하락(0)","상승(1)"]))

In [ ]:
print("10중 분류")
print(classification_report(y_test,reg1))

In [ ]:
X_trainval,X_test,y_trainval,y_test=X[:setlen],X[setlen:],y[:setlen],y[setlen:]
X_train,X_val,y_train,y_val=X[:1500],X[1500:],y[:1500],y[1500:]

reg=GradientBoostingClassifier(random_state=0).fit(X_train,y_train)
y_pred_lower_threshold=reg.decision_function(X_test)>-.8
print(classification_report(y_test,y_pred_lower_threshold,target_names=["하락(0)","상승(1)"]))


In [ ]:

reg1=neighbors.KNeighborsClassifier(n_neighbors=2).fit(X_train,y_train)
reg2=svm.SVC(C=1000,gamma=0.01).fit(X_train,y_train)
reg3=RandomForestClassifier(n_estimators=50).fit(X_train,y_train)
reg4=GradientBoostingClassifier(random_state=0).fit(X_train,y_train)

precision,recall,thresholds=precision_recall_curve(y_test,reg4.decision_function(X_test))

In [ ]:

reg1=neighbors.KNeighborsClassifier(n_neighbors=2).fit(X_train,y_train)
reg2=svm.SVC(C=1000,gamma=0.01).fit(X_train,y_train)
reg3=RandomForestClassifier(n_estimators=50).fit(X_train,y_train)
reg4=GradientBoostingClassifier(random_state=0).fit(X_train,y_train)

precision,recall,thresholds=precision_recall_curve(y_test,reg3.decision_function(X_test))

close_zero=np.argmin(np.abs(thresholds))
plt.plot(precision[close_zero],recall[close_zero],'o',markersize=10,label='0',fillstyle="none",c='k',mew=2)
plt.plot(precision,recall,label="curve")
plt.xlabel("precision")
plt.ylabel('recall')
plt.legend(loc="best")


In [ ]:

# predict_proba
plt.figure(figsize=(12, 12))

precision_rf,recall_rf,thresholds_rf=precision_recall_curve(y_test,reg3.predict_proba(X_test)[:,1])
plt.plot(precision_rf,recall_rf,label="rf")
close_default_rf=np.argmin(np.abs(thresholds_rf-0.5))
plt.plot(precision_rf[close_default_rf],recall_rf[close_default_rf],'^',c='k',markersize=10,label="rf : threshold 0.5",fillstyle="none",mew=2)
plt.xlabel("precision")
plt.ylabel("recall")
plt.legend(loc="best")

precision_svm,recall_svm,thresholds_svm=precision_recall_curve(y_test,reg2.decision_function(X_test))
plt.plot(precision_svm[close_zero],recall_svm[close_zero],'o',markersize=10,label=' svc :threshold 0',fillstyle="none",mew=2)
plt.plot(precision_svm,recall_svm,label="GB")
plt.legend(loc="best")

precision,recall,thresholds=precision_recall_curve(y_test,reg.decision_function(X_test))
plt.plot(precision[close_zero],recall[close_zero],'o',markersize=10,label=' GB :threshold 0',fillstyle="none",c='k',mew=2)
plt.plot(precision,recall,label="GB")
plt.legend(loc="best")


In [ ]:
from sklearn.metrics import average_precision_score
ap_rf=average_precision_score(y_test,reg3.predict_proba(X_test)[:,1])
ap_svc=average_precision_score(y_test,reg2.decision_function(X_test))
ap_GB=average_precision_score(y_test,reg4.decision_function(X_test))
print("랜덤 포레스트의 평균 정밀도 : {:.3f}".format(ap_rf))
print("SVM의 평균 정밀도 : {:.3f}".format(ap_svc))
print("그레이디언트 부스팅의 평균 정밀도 : {:.3f}".format(ap_GB))

In [ ]:
from sklearn.metrics import roc_curve

plt.figure(figsize=(12, 12))
fpr_rf,tpr_rf,thresholds_rf=roc_curve(y_test,reg3.predict_proba(X_test)[:,1])
fpr_svc,tpr_svc,thresholds_svc=roc_curve(y_test,reg2.decision_function(X_test))
fpr_GB,tpr_GB,thresholds_GB=roc_curve(y_test,reg4.decision_function(X_test))
close_default_rf=np.argmin(np.abs(thresholds_rf - 0.5))

plt.plot(fpr_rf,tpr_rf,'k')
plt.plot(fpr_svc,tpr_svc,'b')
plt.plot(fpr_GB,tpr_GB,'yellow')
plt.xlabel("FPR")
plt.ylabel("TPR(recall)")

plt.plot(fpr_svc[close_zero],tpr_svc[close_zero],'o',markersize=10,label="SVC recall : 0 ",fillstyle="none",mew=2)
plt.plot(fpr_rf[close_default_rf],tpr_rf[close_default_rf],'^',markersize=10,label="RF recall : 0.5",fillstyle="none",mew=2)
plt.plot(fpr_GB[close_zero],tpr_GB[close_zero],'^',markersize=10,label="GB",fillstyle="none",mew=2)

In [ ]:
from sklearn.metrics import roc_auc_score
rf_auc=roc_auc_score(y_test,reg3.predict_proba(X_test)[:,1])
svc_auc=roc_auc_score(y_test,reg2.decision_function(X_test))
GB_auc=roc_auc_score(y_test,reg4.decision_function(X_test))
print("랜덤 포레스트의 평균 정밀도 : {:.3f}".format(rf_auc))
print("SVM의 평균 정밀도 : {:.3f}".format(svc_auc))
print("그레이디언트 부스팅의 평균 정밀도 : {:.3f}".format(GB_auc))

In [ ]:
y